In [35]:
import pandas as pd
import pickle
import json
import seaborn as sns
import pprint
import json
import glob
import os
pp = pprint.PrettyPrinter(depth=6)
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = (15.0, 5.0)
pd.set_option('display.max_columns', 120)

In [4]:
szz_folder = "../../szz"

## Szz Unleashed 1 try
* python3 fetch_jira_bugs/fetch.py --issue-code SONAR --jira-project jira.sonarsource.com
* python3 fetch_jira_bugs/git_log_to_array.py --repo-path ../sonarqube --from-commit b326bfd875b0b41
* java -jar build/libs/szz_find_bug_introducers-0.1.jar -i ../issue_list.json -r ../../sonarqube/

In [7]:
szz_files = glob.glob(f"{szz_folder}/szz_unleashed_1/*.json")

### Bug fixing commits

In [19]:
data = {}
for szz_file in szz_files:
    with open(szz_file, "r") as f:
        data.update(json.load(f))

In [25]:
fields =[ 'creationdate',
 'resolutiondate',
 'commitdate',
 'hash']

In [29]:
tuples = []
for key in data.keys():
    inner_tuple = []
    inner_tuple.append(key)
    for field in fields:
        inner_tuple.append(data[key][field])
    tuples.append(tuple(inner_tuple))
issues_fixing_commit = pd.DataFrame(tuples, columns=["issue_name"]+fields)
issues_fixing_commit

,issue_name,creationdate,resolutiondate,commitdate,hash
0,SONAR-13766,2020-08-17 15:43:29 +0200,2020-08-25 16:12:26 +0200,2020-08-17 16:10:03 -0500,60c64b2c9f95bb52d070758fa88a3881ae69b5e7
1,SONAR-12677,2019-11-05 12:25:52 +0100,2019-11-12 15:15:54 +0100,2019-11-06 17:01:34 +0100,8c0b3350b714e835e91d89f958eefa531533070a
2,SONAR-11103,2018-08-02 17:41:21 +0200,2019-01-11 10:26:34 +0100,2019-01-10 15:13:33 +0100,27249ed2abb99bc945cc8b0917f33b50ae96f786
3,SONAR-11467,2018-11-09 15:02:37 +0100,2018-12-05 09:12:34 +0100,2018-12-04 15:30:20 +0100,a08697f54c9f0b98c1a811c22135b9ae303df05e
4,SONAR-11225,2018-09-04 11:19:01 +0200,2019-01-10 17:10:55 +0100,2019-01-10 15:54:51 +0100,355a3b65066e968e3669a97892df81ef15770a55
...,...,...,...,...,...
426,SONAR-6877,2015-09-29 09:06:17 +0200,2015-11-09 10:25:36 +0100,2015-09-29 14:01:50 +0200,e314496d2198c3d827dd7baa7ec1c8fe5a4f6fb1
427,SONAR-12092,2019-05-07 15:08:07 +0200,2019-06-24 14:14:55 +0200,2019-06-21 15:44:00 +0200,36a2571ece01e51f104523ed98f43e6f27f03f75
428,SONAR-6873,2015-09-29 07:34:57 +0200,2015-10-02 13:04:36 +0200,2015-09-28 17:27:35 +0200,3f40e0bcabc43f283e69782dd1b30d7203841188
429,SONAR-11914,2019-04-02 21:07:31 +0200,2020-06-19 16:41:06 +0200,2020-06-19 10:47:02 +0200,b3840f39050b6a805c321748c84726038ccb3f6a


### Bug inducing commits

In [63]:
szz_inducing_folders = list(filter(lambda path: True if os.path.isdir(path) else False, glob.glob(f"{szz_folder}/szz_unleashed_1/results/*")))
szz_inducing_files = list(filter(lambda path: True if (os.path.isfile(path) and ("fix_and_introducers_pairs" in path)) else False, glob.glob(f"{szz_folder}/szz_unleashed_1/results/*")))

In [64]:
szz_inducing_files

['../../szz/szz_unleashed_1/results/fix_and_introducers_pairs.json']

In [67]:
fix_and_introducers_pairs = {}

for file in szz_inducing_files:
    with open(file, "r") as f:
        fix_and_introducers_pairs.update(json.load(f))

for folder in szz_inducing_folders:
    with open(f"{folder}/fix_and_introducers_pairs.json", "r") as f:
        fix_and_introducers_pairs.update(json.load(f))

In [69]:
fix_and_introducers_pairs_tuples = []
for key in fix_and_introducers_pairs.keys():
    fix_and_introducers_pairs_tuples.append((key, fix_and_introducers_pairs[key]))
fix_and_introducers = pd.DataFrame(fix_and_introducers_pairs_tuples, columns=["fixing_commit", "inducing_commit"])

In [76]:
len(set(issues_fixing_commit.hash).intersection(set(fix_and_introducers.fixing_commit)))

373

In [77]:
fix_and_introducers

,fixing_commit,inducing_commit
0,4e162555b40c43254f5b4e76470763bd6e706367,bc9b9edb7c887a74c47c59d615c48ae7024ab392
1,d6393623490fb38fa14c7e3a4a72bbcdbc47b835,1fd5d8abd320428e1e8e4cd1b3ebf793678b10f2
2,18f20eed6127994e54d2fcc7f9772fdcafcfd25d,f6df5fd53b68e63731fdec370778f827cddd6a7b
3,3383b0f376ddbfc348ecedc6fefc72adba929286,d82358c63d0fb979fb3cc27429a42ec833dc161a
4,63a3a432ef8a6127aa690b2bda2abab3c0ff82a5,6c37e7a9c7ef1b048c2067250235e24b386f4fc9
...,...,...
373,cfabfcbc0c4dc65000a2d1aca4212f7b1e720889,fede6d0a2378ce959e7cb1afdf221a92e22b8a6f
374,37915e1548b5a706b0aed606ee3250e3496850b0,e606937326a6f2c9d83d7d2509d73bb237a94356
375,1d2eb3b5ac8a23c5c4aa1b1ede5ce3f24840ce8a,abac7619c39e0e24aa86bb1150ce00b4063fa241
376,73e1d5bb27a020277499fe0be5d059184c59c233,6c37e7a9c7ef1b048c2067250235e24b386f4fc9


In [ ]:
{"4e162555b40c43254f5b4e76470763bd6e706367": "87d444ab3bf860db5d39843d689ff9d330c18384"}